In [1]:
import numpy as np
import pandas as pd
import pandas_profiling
import json
from request_function import request_url

In [2]:
import os

In [3]:
help(request_url)

Help on function request_url in module request_function:

request_url(*param, req='search', form='json', api_url='http://api.duma.gov.ru/api/', token='26d686e46758658adb812b6924d708e99c8a4706', app_token='app61e9cd3433b223ef90ecdcf059b6ccb48338743c', mode=None)
    Function for API GET-request
    
    Parameters
    ----------
    :param param: 
    List of request parameters objects
        list, tuple
    
    :param req: 
    Type of request
        string, default 'search'
    
    :param form:
    Type of data in response
        string, default 'json'
    
    :param api_url, token, app_token:
    Components of requested url
        default 'http://api.duma.gov.ru/api/', API_KEY, APP_KEY
    
    :param mode:
    If 'print' - is printed text form of response
        string, default None



In [4]:
# список всех депутатов и сенаторов
dep_and_sen_all = request_url('', req='deputies')

200 Success!


In [13]:
dep_and_sen_all_extracted = pd.io.json.json_normalize(dep_and_sen_all.json())

In [14]:
dep_and_sen_all_extracted

,id,name,position,isCurrent,factions
0,99111772,Абалаков Александр Николаевич,Депутат ГД,False,"[{'id': '72100004', 'name': 'Фракция Политичес..."
1,99112284,Абасов Мамед Магарамович,Депутат ГД,False,"[{'id': '72100024', 'name': 'Фракция Всероссий..."
2,99100491,Абдулатипов Рамазан Гаджимурадович,Депутат ГД,False,"[{'id': '72100020', 'name': 'Депутатская групп..."
3,99100001,Абдулбасиров Магомедтагир Меджидович,Депутат ГД,False,"[{'id': '72100001', 'name': 'Фракция Аграрной ..."
4,99100002,Абельцев Сергей Николаевич,Депутат ГД,False,"[{'id': '72100005', 'name': 'Фракция Политичес..."
5,99100635,Абраменков Дмитрий Николаевич,Депутат ГД,False,"[{'id': '72100004', 'name': 'Фракция Политичес..."
6,99110156,Абрамов Виктор Семенович,Член СФ,True,"[{'id': '72100011', 'name': 'Депутаты, не вход..."
7,99111774,Абрамов Иван Николаевич,Член СФ,True,"[{'id': '72100005', 'name': 'Фракция Политичес..."
8,99100728,Абрамович Роман Аркадиевич,Депутат ГД,False,"[{'id': '72100011', 'name': 'Депутаты, не вход..."
9,99111998,Аброськин Николай Павлович,Депутат ГД,False,"[{'id': '72100024', 'name': 'Фракция Всероссий..."


## JSON
Результат представляет собой массив объектов, каждый из которых представляет депутата ГД или члена СФ:

**id** — идентификатор депутата ГД или члена СФ

**name** — ФИО

**position** — должность («Депутат ГД» или «Член СФ»)

**isCurrent** — депутат ГД или член СФ в данной должности на данный момент

**factions** — фракция, список, в котором находится словарь либо NaN